In [1]:
import sys
sys.path.append('../../src')

import pandas as pd

from run import run
from utils import save_all_columns

register model: lightgbm
register model: catboost
register model: catboost_class


In [2]:
run(
    in_columns=["anime_id", "user", "user_count_encoding", "text_len_*"],
    target_column="score",
    test_columns=["test_seen_anime_id", "test_seen_user", "test_seen_user_target_encoding", "test_seen_text_len_*"],
    test_id="test_seen_ID",
    run_config={
        "name": "basic-feature-v3",
        "fold-rule":(lambda fold : "seen" + str(fold)),
        "model":"catboost",
        "submit": False,
        "task": "reg",
        "submit_path": "seen.csv",
    },
    model_params={
        "name": "seen-model-reg-v1",
        "iterations": 10000,
        "learning_rate": 0.01,
        "depth": 5,
        "cat_features": ["anime_id", "user"],
    },
    train_params={
        "early_stopping_rounds": 10,
        "verbose": False,
    }
)

[2023-08-26 21:37:39.945031] Check corresponding columns
[2023-08-26 21:37:39.945117] anime_id -> test_seen_anime_id
[2023-08-26 21:37:39.945123] user -> test_seen_user
[2023-08-26 21:37:39.945127] user_count_encoding -> test_seen_user_target_encoding
[2023-08-26 21:37:39.945131] text_len_max -> test_seen_text_len_max
[2023-08-26 21:37:39.945136] text_len_mean -> test_seen_text_len_mean
[2023-08-26 21:37:39.945139] text_len_min -> test_seen_text_len_min
[2023-08-26 21:37:39.945143] text_len_std -> test_seen_text_len_std


wandb: Currently logged in as: abap. Use `wandb login --relogin` to force relogin


load test columns: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 538.71it/s, load test_seen_text_len_std]
/Users/yuchi/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


[2023-08-26 21:37:42.551407] start training
[2023-08-26 21:37:42.573878] start fold: 0
[2023-08-26 21:37:58.532858] start fold: 1
[2023-08-26 21:38:15.127611] start fold: 2
[2023-08-26 21:38:32.374997] start fold: 3
[2023-08-26 21:38:48.348367] save submit file: /Users/yuchi/Desktop/tracon-2023-8/exp/exp04/seen.csv


train/0,█▆▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/1,███▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/2,███▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
val/0,█▆▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/1,████▇▆▆▆▆▆▆▆▆▅▅▄▄▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▁
val/2,███▇▇▇▆▆▆▆▆▆▆▅▅▄▄▄▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▂▁▁
train/0,4.22072
train/1,4.23422
train/2,4.23203
val/0,4.03521
val/1,4.03924


In [3]:
run(
    in_columns=["anime_id", "user"],
    target_column="score",
    test_columns=["test_unseen_anime_id", "test_unseen_user"],
    test_id="test_unseen_ID",
    run_config={
        "name": "basic-feature-v3",
        "fold-rule":(lambda fold : "seen" + str(fold)),
        "model":"catboost",
        "submit": False,
        "task": "reg",
        "submit_path": "unseen.csv",
        "groupkfold": "user_id"
    },
    model_params={
        "name": "seen-model-reg-v1",
        "iterations": 10000,
        "learning_rate": 0.01,
        "depth": 7,
        "cat_features": ["anime_id", "user"],
    },
    train_params={
        "early_stopping_rounds": 10,
        "verbose": False,
    }
)

[2023-08-26 21:38:58.685583] Check corresponding columns
[2023-08-26 21:38:58.686015] anime_id -> test_unseen_anime_id
[2023-08-26 21:38:58.686032] user -> test_unseen_user


load test columns: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 462.11it/s, load test_unseen_user]
/Users/yuchi/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_split.py:909: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


[2023-08-26 21:39:00.513570] start training
[2023-08-26 21:39:04.337587] start fold: 0
[2023-08-26 21:39:06.784540] start fold: 1
[2023-08-26 21:39:11.288896] start fold: 2
[2023-08-26 21:39:14.592231] start fold: 3
[2023-08-26 21:39:20.111443] save submit file: /Users/yuchi/Desktop/tracon-2023-8/exp/exp04/unseen.csv


train/0,█▇▆▅▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/1,██▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
train/3,████▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
val/0,█▇▆▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/1,██▇▇▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val/3,██▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁▁
train/0,4.30086
train/1,4.30626
train/3,4.28705
val/0,4.64316
val/1,4.42104


In [4]:
sample_submission = pd.read_csv('../../data/sample_submission.csv')

In [5]:
seen_pred = pd.read_csv('seen.csv')
unseen_pred = pd.read_csv('unseen.csv')

In [6]:
sample_seen = pd.merge(
    sample_submission, seen_pred, on="ID",  how='left'
)[["ID", "score_y"]].rename(columns={"score_y": "score"})

final_pred = pd.merge(
    sample_seen, unseen_pred, on="ID",  how='left'
)

final_pred["score"] = final_pred["score_x"].fillna(final_pred["score_y"])

In [7]:
label = pd.read_csv('../../data/solution.csv')

In [8]:
label['seen'] = label['ID'].isin(seen_pred['ID'].values)

In [9]:
from sklearn.metrics import mean_squared_error

In [10]:
mean_squared_error(label[label['seen']]['score'], seen_pred['score'])

3.487245725346319

In [11]:
mean_squared_error(label[~label['seen']]['score'], unseen_pred['score'])

4.860182663628279

In [13]:
final_pred[['ID', 'score']].to_csv('result.csv', index=False)

In [14]:
!kaggle competitions submit -c trap-competetion-2023-summer -f result.csv -m ""

alias: -m: unknown option
100%|█████████████████████████████████████████| 842k/842k [00:03<00:00, 278kB/s]
Successfully submitted to traPコンペ 2023 夏